In [4]:
class Value:
    def __init__(self, data, children=(), operation=''):
        self.data = data
        self.prev = children
        self.operation = operation

    def __repr__(self):
        return f'Value(data={self.data})'

    def __add__(self, other):
        return Value(self.data + other.data, (self, other), '+')

    def __add__(self, other):
        return Value(self.data + other.data, (self, other), '+')

def __add__(self, other):
        return Value(self.data + other.data, (self, other), '+')

def __add__(self, other):
        return Value(self.data + other.data, (self, other), '+')

In [5]:
a = Value(2.0)
b = Value(3.0)
c = a + b
c

Value(data=5.0)